In [64]:
import requests
import logging
import re

from urllib.parse import urlencode, parse_qs



class TascConnector:
    
    mission_conf = {
        'gaia' : { 
            'cmd_acro': 'gai',
            'job_acro': 'g',
        },
        'bc' : { 
            'cmd_acro': 'bp',
            'job_acro': 'b',
        }
    }
    
    common_params = {
     'queryType': ['run'],
     'ops': ['ops_Routine'],
     'scenarioId': ['NEW_ID'],
     'inputCase': ['DEFAULT_case'],
     'opsConfig': ['opsSelect'],
     'jobConfig': ['jobTable'],
     'scenarioId': ['NEW_ID'],
     'inputCase': ['DEFAULT_case']
    }
    
    
    attitude_params = {
     'queryType': ['run'],
     'ops': ['ops_Routine'],
     'job': ['g_job_tmpl_fdtool_crttab_quat.dat'],
     'opsConfig': ['opsSelect'],
     'jobConfig': ['jobTable'],
     'TIME_SCALE': ['TDB'],
     'INITIAL_TIME': ['2023/06/10 00:00:00.000'],
     'FINAL_TIME': ['2023/06/11 00:00:00.000'],
     'TIME_STEP': ['000 00:01:00.000'],
     'FRAME': ['mean equatorial J2000'],
     'scenarioId': ['NEW_ID'],
     'inputCase': ['DEFAULT_case']
    }
    
    state_vector_params = {
        'queryType': ['run'],
 'ops': ['ops_Routine'],
 'job': ['g_job_tmpl_fdtool_crttab_state.dat'],
 'opsConfig': ['opsSelect'],
 'jobConfig': ['jobTable'],
 'TIME_SCALE': ['TDB'],
 'INITIAL_TIME': ['2023/06/10 00:00:00.000'],
 'FINAL_TIME': ['2023/06/11 00:00:00.000'],
 'TIME_STEP': ['000 01:00:00.000'],
 'OBJECT': ['Gaia'],
 'REF_OBJECT': ['Solar System Barycentre'],
 'FRAME': ['mean equatorial J2000'],
 'LTC_FLAG': ['NO'],
 'scenarioId': ['NEW_ID'],
 'inputCase': ['DEFAULT_case']}


    
    
    def __init__(self, url='https://www.fd-tasc.info'):
        self.base_url = url
        
    def get_attitude(self, mission):
        
        config = self.mission_conf.get(mission)
        params = self.attitude_params
        
        self.__set_mission_params(params, config, 'quat')
        
        url = self.base_url + f'/{config.get("cmd_acro")}cmd-cgi-bin/seqgenExec.pl'
        content = self.__get_url(url, params)
        print(self.__extract_pre(content)[0])

    def get_attitude(self, mission):
        
        config = self.mission_conf.get(mission)
        params = self.attitude_params
        
        self.__set_mission_params(params, config, 'quat')
        
        url = self.base_url + f'/{config.get("cmd_acro")}cmd-cgi-bin/seqgenExec.pl'
        content = self.__get_url(url, params)
        print(self.__extract_pre(content)[0])
        
        
    def __get_url(self, url, params):
        response = requests.get(url, params)
        if response.status_code == 200:
            return response.content.decode('utf-8')
        else:
            raise ValueError(f'{response.status_code}')
    
    def __extract_pre(self, txt):
        
        return re.findall("<pre>(.*?)</pre>", txt, re.DOTALL)
    
    
    def __set_mission_params(self, params, config, operation):
        params['job'] = f'{config.get("job_acro")}_job_tmpl_fdtool_crttab_{operation}.dat'
        
        

In [66]:
tc = TascConnector()
# tc.get_attitude('gaia')
# tc.get_attitude('bc')

In [68]:
url = 'queryType=run&ops=ops_Routine&job=g_job_tmpl_fdtool_crttab_state.dat&opsConfig=opsSelect&jobConfig=jobTable&TIME_SCALE=TDB&INITIAL_TIME=2023%2F06%2F10+00%3A00%3A00.000&FINAL_TIME=2023%2F06%2F11+00%3A00%3A00.000&TIME_STEP=000+01%3A00%3A00.000&OBJECT=Gaia&REF_OBJECT=Solar+System+Barycentre&FRAME=mean+equatorial+J2000&LTC_FLAG=NO&scenarioId=NEW_ID&inputCase=DEFAULT_case'

parse_qs(url)

{'queryType': ['run'],
 'ops': ['ops_Routine'],
 'job': ['g_job_tmpl_fdtool_crttab_state.dat'],
 'opsConfig': ['opsSelect'],
 'jobConfig': ['jobTable'],
 'TIME_SCALE': ['TDB'],
 'INITIAL_TIME': ['2023/06/10 00:00:00.000'],
 'FINAL_TIME': ['2023/06/11 00:00:00.000'],
 'TIME_STEP': ['000 01:00:00.000'],
 'OBJECT': ['Gaia'],
 'REF_OBJECT': ['Solar System Barycentre'],
 'FRAME': ['mean equatorial J2000'],
 'LTC_FLAG': ['NO'],
 'scenarioId': ['NEW_ID'],
 'inputCase': ['DEFAULT_case']}